## 1. Two Sum (Easy)

Given an array of integers `nums` and an integer `target`, return indices of the two numbers such that they add up to `target`.

You may assume that each input would have ***exactly one solution***, and you may not use the same element twice.

You can return the answer in any order


**Example 1:**
```
    Input: nums = [2,7,11,15], target =9
    Output: [0, 1]
    Output: Because nums[0] + nums[1] == 9, we return [0, 1]
```
**Example 2:**
```
    Input: nums = [3,2,4], target = 6
    Output: [1,2]
```
**Example 3:**
```
    Input: nums = [3,3], target = 6
    Output: [0,1]
```
**Constraints:**
```
* 2 <= nums.length <= 103
* -109 <= nums[i] <= 109
* -109 <= target <= 109
*Only one valid answer exists.
```

In [1]:
class Solution:
    def twoSum(self, nums, target):
        map = dict()
        for i in range(len(nums)):
            complement = target - nums[i]
            if complement in map:
                return [map[complement], i]
            map[nums[i]] = i


## 146. LRU Cache (Medium)

Design a data Structure that follows the constraints of a Least Recently Used (LRU) cache.

Implement `LRUCache` class:
* `LRUCache(int capacity)` Initialize the LRU cache with positive size `capacity`
* `int get(int key)` Return the value of the `key` if the key exists, otherwise return -1
* `void put(int key, int value)` Update the value of the `key` if `key` exists. Otherwise, add the `key-value` pair to the cache. If the number of keys exceeds the `capacity` form this operation, **evict** the least recently used key.

**Follow up:**
Could you do `get` and `put` in `O(1)` time complexity?

**Example 1:**

```
Input
["LRUCache", "put", "put", "get", "put", "get", "put", "get", "get", "get"]
[[2], [1, 1], [2, 2], [1], [3, 3], [2], [4, 4], [1], [3], [4]]
Output
[null, null, null, 1, null, -1, null, -1, 3, 4]

Explanation
LRUCache lRUCache = new LRUCache(2);
lRUCache.put(1, 1); // cache is {1=1}
lRUCache.put(2, 2); // cache is {1=1, 2=2}
lRUCache.get(1);    // return 1
lRUCache.put(3, 3); // LRU key was 2, evicts key 2, cache is {1=1, 3=3}
lRUCache.get(2);    // returns -1 (not found)
lRUCache.put(4, 4); // LRU key was 1, evicts key 1, cache is {4=4, 3=3}
lRUCache.get(1);    // return -1 (not found)
lRUCache.get(3);    // return 3
lRUCache.get(4);    // return 4
```
**Constraints:**
* 1 <= capacity <= 3000
* 0 <= key <= 3000
* 0 <= value <= 104
* At most 3 * 104 calls will be made to get and put.

In [2]:
# OrderedDict
from collections import OrderedDict
class LRUCache(OrderedDict):

    def __init__(self, capacity):
        """
        :param capacity: int
        """
        self.capacity = capacity

    def get(self, key):
        """
        :param key: int
        :return: int
        """
        if key not in self:
            return -1
        self.move_to_end(key)
        return self[key]

    def put(self, key, value):
        """
        :param key: int
        :param value: int
        :return: void
        """
        if key in self:
            self.move_to_end(key)
        self[key] = value
        if len(self) > self.capacity:
            self.popitem(last=False)

In [3]:
# Hashmap + DoubleLinkedList
class DLinkedNode():
    def __init__(self):
        self.key = 0
        self.value = 0
        self.prev = None
        self.next = None

class LRUCache():
    def _add_node(self, node):
        """
        Always add the new node right after head.
        :param node:
        :return:
        """
        node.prev = self.head
        node.next = self.head.next

        self.head.next.prev = node
        self.head.next = node

    def _remove_node(self, node):
        """
        Remove an existing node from the linked list
        :param node:
        :return:
        """
        prev = node.prev
        new = node.next

        prev.next = new
        new.prev = prev

    def _move_to_head(self, node):
        """
        Move Certain node in between to the head.
        :param node:
        :return:
        """
        self._remove_node(node)
        self._add_node(node)

    def _pop_tail(self):
        """
        Pop the current tail
        :return:
        """
        res = self.tail.prev
        self._remove_node(res)
        return res

    def __init__(self, capacity: int):
        """
        :param capacity: int
        """
        self.cache = {}
        self.size = 0
        self.capacity = capacity
        self.head, self.tail = DLinkedNode(), DLinkedNode()

        self.head.next = self.tail
        self.tail.prev = self.head

    def get(self, key):
        """
        :param key: int
        :return: int
        """
        node = self.cache.get(key, None)
        if not node:
            return -1
        self._move_to_head(node)
        return node.value

    def put(self, key, value):
        """
        :param key: int
        :param value: int
        :return: None
        """
        node = self.cache.get(key)

        if not node:
            newNode = DLinkedNode()
            newNode.key = key
            newNode.value = value

            self.cache[key] = newNode
            self._add_node(newNode)
            self.size += 1

            if self.size > self.capacity:
                tail = self._pop_tail()
                del self.cache[tail.key]
                self.size -= 1
        else:
            node.value = value
            self._move_to_head(node)


## 42. Trapping Rain Water (Hard)

Given `n` non-negative integers representing an elevation map where the width of each bar is `l`, compute how much water it can trap after raining

**Example 1:**
```
Input: height = [0,1,0,2,1,0,1,3,2,1,2,1]
Output: 6
Explanation: The above elevation map (black section) is represented by array [0,1,0,2,1,0,1,3,2,1,2,1]. In this case, 6 units of rain water (blue section) are being trapped.
```
**Example 2:**
```
Input: height = [4,2,0,3,2,5]
Output: 9
```
**Constraints:**
* n == height.length
* 0 <= n <= 3 * 104
* 0 <= height[i] <= 105

In [4]:
# Dynamic Programming:
"""
1. Find maximum height of bar from the left end upto an index i in the array `left_max`
2. Find maximum height of bar from the right end upto an index i in the array `right_max`
3. Iterate over height array and update ans: Add min(left_max[i], right_max[i]) - height[i] to ans
Time Complexity: O(n), Space Complexity: O(n) extra space
"""

class Solution:
    def trap(self, height):
        """
        :param height: List[int]
        :return: int
        """
        if not height:
            return 0
        ans = 0
        size = len(height)
        left_max, right_max = [0] * size, [0] * size

        left_max[0] = height[0]
        for i in range(1, size):
            left_max[i] = max(height[i], left_max[i-1])

        right_max[size-1] = height[size-1]
        for i in reversed(range(size-1)):
            right_max[i] = max(height[i], right_max[i+1])

        for i in range(1, size-1):
            ans +=  min(left_max[i], right_max[i]) - height[i]
        return ans

In [1]:
# Using Stacks
"""
We keep a stack and iterate over the array.
We add the index of the bar to the stack if bar <= top of stack, which mean the current bar is bounded by the previous bar in the stack
If we found a current bar > the top,  we are sure that the bar at the top of the stack is bounded by the current bar and previous bar in the stack.
Hence, we can pop it and add resulting trapped water to `ans`
Time Complexity: O(n), Space Complexity: O(n)
"""
class Solution:
    def trap(self, height):
        """
        :param height: List[int]
        :return: int
        """
        ans, current = 0, 0
        st = []
        while (current < len(height)):
            while (st and height[current]>height[st[-1]]):
                top = st.pop()
                if len(st) == 0:
                    break
                distance = current - st[-1] - 1
                bounded_height = min(height[current], height[st[-1]]) - height[top]
                ans += distance * bounded_height
            st.append(current)
            current+=1
        return ans

In [6]:
# Using 2 Pointers
"""
If there is a larger bar at one end (say right), we are assured that the water trapped would be dependant on height of bar
in current direction (from left ot right).
As soon as we find the bar at other end (right) is smaller, we start iterating in opposite direction (from right to left).
We must maintain left_max and right_max during the iteration, but now we can do it in one iteration using 2 pointers, switching between two
Time complexity: O(n), Space complexity: O(1)
"""
class Solution:
    def trap(self, height):
        """
        :param height: List[int]
        :return: int
        """
        left, right = 0, len(height)-1
        ans = 0
        left_max, right_max = 0, 0
        while left < right:
            if height[left] < height[right]:
                if height[left] > left_max:
                    left_max = height[left]
                else:
                    ans += left_max - height[left]
                left += 1
            else:
                if height[right] > right_max:
                    right_max = height[right]
                else:
                    ans += right_max - height[right]
                right -= 1
        return ans

## 200. Number of Islands (Medium)
Given an `mxn` 2D binary grid `grid` which represents a map of `'1'`s (land) and `'0'`s (water), return the number of islands.

An **island** is surrounded by water and is formed by connecting adjacent lands horizontally or vertically. You may assume all four edges of the grid are all surrounded by water

**Example 1:**
```
Input: grid = [
  ["1","1","1","1","0"],
  ["1","1","0","1","0"],
  ["1","1","0","0","0"],
  ["0","0","0","0","0"]
]
Output: 1
```
**Example 2:**
```
Input: grid = [
  ["1","1","0","0","0"],
  ["1","1","0","0","0"],
  ["0","0","1","0","0"],
  ["0","0","0","1","1"]
]
Output: 3
```

**Constraints:**
* m == grid.length
* n == grid[i].length
* 1 <= m, n <= 300
* grid[i][j] is '0' or '1'.

In [4]:
# DFS (Depth First Search): using recursive function
# Time Complexity: O(MxN), Space Complexity: O(MxN)
class Solution:
    def dfs(self, grid, r, c):
        nr = len(grid)
        nc = len(grid[0])
        if r < 0 or c < 0 or r >= nr or c >= nc or grid[r][c] == '0':
            return
        grid[r][c]='0'
        self.dfs(grid, r-1, c)
        self.dfs(grid, r+1, c)
        self.dfs(grid, r, c-1)
        self.dfs(grid, r, c+1)
    def numIslands(self, grid):
        """
        :param grid: List[List[str]
        :return: int
        """
        if len(grid) == 0:
            return 0
        nr = len(grid)
        nc = len(grid[0])
        num_islands = 0
        for r in range(nr):
            for c in range(nc):
                if grid[r][c] == '1':
                    num_islands += 1
                    self.dfs(grid, r, c)
        return num_islands

In [5]:
# BFS (Breadth First Search): Using Queue
# Time Complexity: O(MxN), Space Complexity: O(min(M,N))

class Solution:
    def numIslands(self, grid):
        if len(grid) == 0:
            return 0
        nr = len(grid)
        nc = len(grid[0])
        num_islands=0
        for r in range(nr):
            for c in range(nc):
                if grid[r][c]=='1':
                    num_islands +=1
                    neighbors = []
                    # neighbors.append((r,c))
                    neighbors.append(r*nc + c)
                    while neighbors:
                        # rc = neighbors.pop(0)
                        # row, col = rc[0], rc[1]
                        id = neighbors.pop(0)
                        row, col = id // nc, id % nc
                        if row - 1 >= 0 and grid[row-1][col] == '1':
                            neighbors.append((row-1)*nc + col)
                            grid[row-1][col] = '0'
                        if row + 1 < nr and grid[row+1][col] == '1':
                            neighbors.append((row+1)*nc + col)
                            grid[row+1][col] = '0'
                        if col - 1 >= 0 and grid[row][col-1] == '1':
                            neighbors.append(row*nc + col-1)
                            grid[row][col-1] = '0'
                        if col + 1 < nc and grid[row][col+1] == '1':
                            neighbors.append(row*nc + col+1)
                            grid[row][col+1] = '0'
        return num_islands

## 56. Merge Intervals (Medium)
Given an array of `intervals` where ```intervals[i] = [start_i, end_i]```


